In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
import scanpy as sc
import jax
import os
from cfp.metrics import compute_metrics, compute_mean_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp
import anndata as ad
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

from gears import PertData

In [202]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]

In [203]:
split = 0
seed = split + 1

##### Load data

In [204]:
DATA_DIR = "/home/haicu/soeren.becker/repos/ot_pert_reproducibility/norman2019/norman_preprocessed_adata"

adata_train_path = os.path.join(DATA_DIR, f"adata_train_pca_50_split_{split}.h5ad")
adata_test_path = os.path.join(DATA_DIR, f"adata_val_pca_50_split_{split}.h5ad")
adata_ood_path = os.path.join(DATA_DIR, f"adata_test_pca_50_split_{split}.h5ad")

# load data splits
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

# remove ctrl from ood and val
print(adata_ood.shape, adata_test.shape)
adata_ood = adata_ood[adata_ood.obs.condition != "ctrl"]
adata_test = adata_test[adata_test.obs.condition != "ctrl"]
print(adata_ood.shape, adata_test.shape)

(29254, 5045) (11254, 5045)
(28754, 5045) (10754, 5045)


In [206]:
data_dir = "/home/haicu/soeren.becker/repos/ot_pert_reproducibility/norman2019"
pert_data = PertData(
    data_dir,  
    gene_set_path=os.path.join(data_dir, "essential_norman.pkl")
)
pert_data.load(data_path = data_dir)

Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['RHOXF2BB+ctrl' 'LYL1+IER5L' 'ctrl+IER5L' 'KIAA1804+ctrl' 'IER5L+ctrl'
 'RHOXF2BB+ZBTB25' 'RHOXF2BB+SET']
Local copy of pyg dataset is detected. Loading...
Done!


In [211]:
pert_data.prepare_split(split = "simulation", seed = seed)

Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:9
combo_seen1:43
combo_seen2:19
unseen_single:36
Done!


here1


In [215]:
sorted(pert_data.set2conditions["train"])

['AHR+FEV',
 'AHR+ctrl',
 'ATL1+ctrl',
 'BAK1+ctrl',
 'BPGM+SAMD1',
 'BPGM+ctrl',
 'CBFA2T3+ctrl',
 'CEBPA+ctrl',
 'CEBPE+CEBPA',
 'CEBPE+PTPN12',
 'CEBPE+RUNX1T1',
 'CEBPE+ctrl',
 'CLDN6+ctrl',
 'CNN1+UBASH3A',
 'CNN1+ctrl',
 'COL1A1+ctrl',
 'CSRNP1+ctrl',
 'DLX2+ctrl',
 'ETS2+IGDCC3',
 'ETS2+IKZF3',
 'ETS2+ctrl',
 'FEV+MAP7D1',
 'FEV+ctrl',
 'FOSB+UBASH3B',
 'FOSB+ctrl',
 'FOXA1+FOXF1',
 'FOXA1+ctrl',
 'FOXA3+FOXA1',
 'FOXA3+FOXF1',
 'FOXA3+ctrl',
 'FOXF1+HOXB9',
 'FOXF1+ctrl',
 'FOXO4+ctrl',
 'GLB1L2+ctrl',
 'HES7+ctrl',
 'HK2+ctrl',
 'HNF4A+ctrl',
 'HOXA13+ctrl',
 'HOXB9+ctrl',
 'HOXC13+ctrl',
 'IGDCC3+MAPK1',
 'IGDCC3+ZBTB25',
 'IGDCC3+ctrl',
 'IKZF3+ctrl',
 'IRF1+SET',
 'IRF1+ctrl',
 'ISL2+ctrl',
 'KIF18B+KIF2C',
 'KIF18B+ctrl',
 'KIF2C+ctrl',
 'KMT2A+ctrl',
 'LHX1+ctrl',
 'LYL1+ctrl',
 'MAML2+ctrl',
 'MAP2K3+MAP2K6',
 'MAP2K3+SLC38A2',
 'MAP2K3+ctrl',
 'MAP2K6+IKZF3',
 'MAP2K6+ctrl',
 'MAP4K3+ctrl',
 'MAP4K5+ctrl',
 'MAP7D1+ctrl',
 'MAPK1+IKZF3',
 'MAPK1+TGFBR2',
 'MAPK1+ctrl',


In [216]:
sorted(pert_data.set2conditions["val"])

['ARRDC3+ctrl',
 'BPGM+ZBTB1',
 'C19orf26+ctrl',
 'CITED1+ctrl',
 'CNNM4+ctrl',
 'ELMSAN1+ctrl',
 'ETS2+MAP7D1',
 'FEV+CBFA2T3',
 'FOSB+IKZF3',
 'FOSB+PTPN12',
 'KLF1+BAK1',
 'KLF1+CEBPA',
 'KLF1+CLDN6',
 'KLF1+MAP2K6',
 'KLF1+TGFBR2',
 'KLF1+ctrl',
 'LHX1+ELMSAN1',
 'MAP2K3+ELMSAN1',
 'MAP2K6+ELMSAN1',
 'SAMD1+ZBTB1',
 'SET+KLF1',
 'SGK1+TBX2',
 'TBX2+ctrl',
 'TGFBR2+C19orf26',
 'TGFBR2+ETS2',
 'ZBTB1+ctrl',
 'ctrl+C19orf26',
 'ctrl+ELMSAN1',
 'ctrl+KLF1',
 'ctrl+TBX2',
 'ctrl+ZBTB1']

In [ ]:
KLF1

In [212]:
dir(pert_data)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'adata',
 'create_cell_graph',
 'create_cell_graph_dataset',
 'create_dataset_file',
 'ctrl_adata',
 'data_path',
 'dataset_name',
 'dataset_path',
 'dataset_processed',
 'default_pert_graph',
 'gene2go',
 'gene_names',
 'gene_set_path',
 'get_dataloader',
 'get_pert_idx',
 'load',
 'new_data_process',
 'node_map',
 'node_map_pert',
 'pert_names',
 'prepare_split',
 'seed',
 'set2conditions',
 'set_pert_genes',
 'split',
 'subgroup',
 'train_gene_set_size']

##### Doublecheck that we are using the same splits as GEARS

In [205]:
path_to_splits_gears = f'/lustre/groups/ml01/workspace/leander.dony/projects/cellflow/241106_gears/reproduce_biolord_repro/data/perturbations/norman/norman2019/splits/norman2019_simulation_{seed}_0.75_subgroup.pkl'
subgroups_gears = pd.read_pickle(path_to_splits_gears)
test_conditions_gears, ood_conditions_gears = [], []
for subgroup in subgroups_gears["test_subgroup"].keys():
    ood_conditions_gears.extend(subgroups_gears["test_subgroup"][subgroup])
ood_conditions_gears = np.array(ood_conditions_gears)

for subgroup in subgroups_gears["val_subgroup"].keys():
    test_conditions_gears.extend(subgroups_gears["val_subgroup"][subgroup])
test_conditions_gears = np.array(test_conditions_gears)

test_conditions = np.asarray(adata_test.obs.condition.values.unique())

assert len(test_conditions[~np.isin(test_conditions, test_conditions_gears)]) == 0
assert len(test_conditions_gears[~np.isin(test_conditions_gears, test_conditions)]) == 0

ood_conditions = np.asarray(adata_ood.obs.condition.values.unique())
assert len(ood_conditions[~np.isin(ood_conditions, ood_conditions_gears)]) == 0
assert len(ood_conditions_gears[~np.isin(ood_conditions_gears, ood_conditions)]) == 0

assert np.isin(subgroups_gears["test_subgroup"]['combo_seen2'], adata_ood.obs.condition).all()

print("Assertions passed -> We are using the same splits as GEARS.")

Assertions passed -> We are using the same splits as GEARS.


In [107]:
# compute pca on full dataset
adata_all = ad.concat((adata_train, adata_test, adata_ood))
cfpp.centered_pca(adata_all, n_comps=10)

In [158]:
adata_train_ctrl = adata_train[adata_train.obs.condition == "ctrl"].copy()
adata_train_single = adata_train[adata_train.obs.kategory == "single"].copy()
adata_ood_double_seen_2 = adata_ood[adata_ood.obs.condition.isin(subgroups_gears["test_subgroup"]['combo_seen2'])].copy()

In [166]:
genes = adata_train_single.obs.condition.str.split("+", expand=True).values
genes_1 = genes[:, 0]
print(genes_1)
genes_2 = genes[:, 1]
mask = genes_1 == "ctrl"

genes_1[mask], genes_2[mask] = genes_2[mask], genes_1[mask]

['TSC22D1' 'MAML2' 'ctrl' ... 'ctrl' 'FOXA3' 'MAP4K3']


In [167]:
# make sure that the perturbed gene is always gene 1
adata_train_single.obs.loc[:, "condition_ordered"] = genes_1 + "+" + genes_2
adata_train_single.obs.loc[:, "gene_1_ordered"] = genes_1
adata_train_single.obs.loc[:, "gene_2_ordered"] = genes_2

In [178]:
cells_1.shape, cells_2.shape

((0, 5045), (761, 5045))

In [180]:
gene_1

'KLF1'

In [201]:
adata_test[adata_test.obs.kategory == "single"].obs.condition.unique().tolist()

['ctrl+ELMSAN1',
 'KLF1+ctrl',
 'ctrl+KLF1',
 'ctrl+TBX2',
 'C19orf26+ctrl',
 'ZBTB1+ctrl',
 'ARRDC3+ctrl',
 'ctrl+ZBTB1',
 'ctrl+C19orf26',
 'CNNM4+ctrl',
 'TBX2+ctrl',
 'ELMSAN1+ctrl',
 'CITED1+ctrl']

In [193]:
np.unique(
    adata_test.obs.gene_1.unique().tolist(),
    adata_test.obs.gene_2.unique().tolist(),
)

(array(['ARRDC3', 'BPGM', 'C19orf26', 'CITED1', 'CNNM4', 'ELMSAN1', 'ETS2',
        'FEV', 'FOSB', 'KLF1', 'LHX1', 'MAP2K3', 'MAP2K6', 'SAMD1', 'SET',
        'SGK1', 'TBX2', 'TGFBR2', 'ZBTB1', 'ctrl'], dtype='<U8'),
 array([ 6, 19,  3, 17, 11, 13, 15, 16, 18,  0,  4, 10,  8,  9,  7, 14, 12,
         1,  5,  2]))

In [184]:
adata_ood_double_seen_2.obs.condition.unique()

['UBASH3B+PTPN12', 'KLF1+FOXA1', 'TBX3+TBX2', 'CEBPE+KLF1', 'UBASH3B+ZBTB25', ..., 'ETS2+MAPK1', 'FEV+ISL2', 'POU3F2+CBFA2T3', 'FOXA1+HOXB9', 'SNAI1+DLX2']
Length: 19
Categories (19, object): ['AHR+KLF1', 'CEBPE+CNN1', 'CEBPE+KLF1', 'CNN1+MAPK1', ..., 'TBX3+TBX2', 'UBASH3B+PTPN12', 'UBASH3B+ZBTB25', 'ZC3HAV1+HOXC13']

In [176]:
NUM_SAMPLED_CELLS = 500

In [177]:
all_predictions, all_conditions = [], []

for condition in tqdm(adata_ood_double_seen_2.obs.condition.unique()):

    # get perturbed genes
    gene_1, gene_2 = condition.split("+")

    # get perturbed gene expression profiles
    cells_1 = adata_train_single[adata_train_single.obs.gene_1_ordered == gene_1].X
    cells_2 = adata_train_single[adata_train_single.obs.gene_1_ordered == gene_2].X

    # choose NUM_SAMPLED_CELLS cells
    random_idcs_1 = np.random.choice(cells_1.shape[0], size=NUM_SAMPLED_CELLS, replace=True)
    random_idcs_2 = np.random.choice(cells_2.shape[0], size=NUM_SAMPLED_CELLS, replace=True)
    
    cells_1 = cells_1[random_idcs_1].todense()
    cells_2 = cells_2[random_idcs_2].todense()

    # get control cells
    random_idcs_ctrl = np.random.choice(adata_train_ctrl.shape[0], size=NUM_SAMPLED_CELLS, replace=True)
    ctrl_cells = adata_train_ctrl.X[random_idcs_ctrl]

    # compute displacements
    displacement_1 = cells_1 - ctrl_cells
    displacement_2 = cells_2 - ctrl_cells

    # get predictions
    predictions = np.asarray(ctrl_cells + displacement_1 + displacement_2)
    all_predictions.append(predictions)
    all_conditions.extend([condition] * NUM_SAMPLED_CELLS)

  0%|          | 0/19 [00:00<?, ?it/s]

ValueError: a must be greater than 0 unless no samples are taken

In [84]:
adata_pred_ood  = ad.AnnData(X=np.vstack(all_predictions), obs=pd.DataFrame(all_conditions, columns=["condition"]))

/home/haicu/soeren.becker/miniconda3/envs/env_cfp2/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [85]:
adata_pred_ood

AnnData object with n_obs × n_vars = 7500 × 5045
    obs: 'condition'

In [100]:
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_all)
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_all)

ood_data_target_encoded, ood_data_target_decoded = {}, {}
ood_data_target_encoded_predicted, ood_data_target_decoded_predicted = {}, {}

subgroups = ["double_seen_2"]

for subgroup in subgroups:

    ood_data_target_encoded[subgroup] = {}
    ood_data_target_decoded[subgroup] = {}
    ood_data_target_encoded_predicted[subgroup] = {}
    ood_data_target_decoded_predicted[subgroup] = {}
    
    for cond in adata_ood.obs["condition"].cat.categories:
        if cond == "ctrl":
            continue
        
        select = adata_ood.obs["condition"] == cond
        select2 = adata_pred_ood.obs["condition"] == cond
        if subgroup != "all":
            select = select & (adata_ood.obs.subgroup == subgroup)

        if not any(select):
            # the condition is not part of this subgroup
            continue
        
        # pca space
        ood_data_target_encoded[subgroup][cond] = adata_ood[select].obsm["X_pca"]
        ood_data_target_encoded_predicted[subgroup][cond] = adata_pred_ood[select2].obsm["X_pca"]
        # print(ood_data_target_encoded[subgroup][cond].shape, ood_data_target_encoded_predicted[subgroup][cond].shape)
    
        # gene space
        ood_data_target_decoded[subgroup][cond] = np.asarray(adata_ood[select].X.todense())
        ood_data_target_decoded_predicted[subgroup][cond] = adata_pred_ood[select2].X
        # print(ood_data_target_decoded[subgroup][cond].shape, ood_data_target_decoded_predicted[subgroup][cond].shape)

In [103]:
ood_metrics_encoded, mean_ood_metrics_encoded = {}, {}
ood_metrics_decoded, mean_ood_metrics_decoded = {}, {}
deg_ood_metrics, deg_mean_ood_metrics = {}, {}
ood_deg_dict = {}
ood_deg_target_decoded_predicted, ood_deg_target_decoded = {}, {}

for subgroup in tqdm(subgroups[::-1]):

    print(f"subgroup: {subgroup}")

    print("Computing ood_metrics_encoded")
    # ood set: evaluation in encoded (=pca) space
    ood_metrics_encoded[subgroup] = jax.tree_util.tree_map(
        compute_metrics, 
        # compute_metrics_fast, 
        ood_data_target_encoded[subgroup], 
        ood_data_target_encoded_predicted[subgroup]
    )
    mean_ood_metrics_encoded[subgroup] = compute_mean_metrics(
        ood_metrics_encoded[subgroup], 
        prefix="encoded_ood_",
    )

    print("Computing ood_metrics_decoded")
    # ood set: evaluation in decoded (=gene) space
    ood_metrics_decoded[subgroup] = jax.tree_util.tree_map(
        # compute_metrics, 
        compute_metrics_fast, 
        ood_data_target_decoded[subgroup], 
        ood_data_target_decoded_predicted[subgroup]
    )
    mean_ood_metrics_decoded[subgroup] = compute_mean_metrics(
        ood_metrics_decoded[subgroup], 
        prefix="decoded_ood_",
    )

    # ood set
    ood_deg_dict[subgroup] = {
        k: v
        for k, v in adata_train.uns['rank_genes_groups_cov_all'].items() 
        if k in ood_data_target_decoded_predicted[subgroup].keys()
    }

    print("Apply DEG mask")
    # ood set
    ood_deg_target_decoded_predicted[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded_predicted[subgroup], 
        ood_deg_dict[subgroup]
    )
    
    ood_deg_target_decoded[subgroup] = jax.tree_util.tree_map(
        get_mask, 
        ood_data_target_decoded[subgroup], 
        ood_deg_dict[subgroup]
    )

    print("Compute metrics on DEG subsetted decoded")
    deg_ood_metrics[subgroup] = jax.tree_util.tree_map(
        compute_metrics, 
        # compute_metrics_fast, 
        ood_deg_target_decoded[subgroup], 
        ood_deg_target_decoded_predicted[subgroup]
    )
    deg_mean_ood_metrics[subgroup] = compute_mean_metrics(
        deg_ood_metrics[subgroup], 
        prefix="deg_ood_"
    )

  0%|          | 0/1 [00:00<?, ?it/s]

subgroup: double_seen_2
Computing ood_metrics_encoded
Computing ood_metrics_decoded
Apply DEG mask
Compute metrics on DEG subsetted decoded


In [104]:
deg_mean_ood_metrics

{'double_seen_2': {'deg_ood_r_squared': 0.9708234429359436,
  'deg_ood_sinkhorn_div_1': 32.74098542531331,
  'deg_ood_sinkhorn_div_10': 11.421825281778972,
  'deg_ood_sinkhorn_div_100': 2.0428421020507814,
  'deg_ood_e_distance': np.float64(3.3670727565598644),
  'deg_ood_mmd': np.float32(0.02222708)}}

In [105]:
collected_results = {
    "ood_metrics_encoded": ood_metrics_encoded,
    "mean_ood_metrics_encoded": mean_ood_metrics_encoded,
    "ood_metrics_decoded": ood_metrics_decoded,
    "mean_ood_metrics_decoded": mean_ood_metrics_decoded,
    "deg_ood_metrics": deg_ood_metrics,
    "deg_mean_ood_metrics": deg_mean_ood_metrics,
    "ood_deg_dict": ood_deg_dict,
    "ood_deg_target_decoded_predicted": ood_deg_target_decoded_predicted,
    "ood_deg_target_decoded": ood_deg_target_decoded,
}

In [109]:
OUT_DIR = "/lustre/groups/ml01/workspace/ot_perturbation/data/norman_soren/additive"
os.makedirs(OUT_DIR, exist_ok=True)
pd.to_pickle(collected_results, os.path.join(OUT_DIR, f"norman_additive_split_{split}_collected_results.pkl"))